# Table of Contents


In [1]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
#plt.style.use('ggplot')
import ipywidgets as widgets
import sys, os, io, string, shutil, math
from hublib.ui import Submit
from hublib.ui import RunCommand
import hublib.use
%use boost-1.62.0-mpich2-1.3-gnu-4.7.2
#%use lammps-31Mar17
import numpy as np
import re
import time
from threading import Thread
from io import StringIO
from ipywidgets import Layout, Box, Label, Output
from IPython.display import display,HTML
sys.path.append('../python/')
#import train as nn
from dump import dump
from xyz import xyz
from svg import svg
import random
#from IPython.core.display import display, HTML
display(HTML("<style>.container { width:1450px !important; }</style>"))

style = {'description_width': 'initial'}

left_column='340px'
right_column='940px'
app_width='1280px'
image_width=540

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    padding='5px'
)

form_item_layout_slider_text = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    padding='5px'
)

box_layout_image=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    #width='25%',
    padding='5px',
    #min_height='20px',
    #min_width='200px'
    width=left_column,
    height='304.5px'
)
box_layout_phisical=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    #width='25%',
    padding='5px',
    #min_height='20px',
    #min_width='200px'
    width=left_column,
    #height='180px'
)
box_layout_computing=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=left_column,
    #height='103px'
)
box_layout_progress=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=left_column,
    ##height='55px'
)

box_layout_output=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=left_column,
    #height='150px'
)
box_layout_about=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=right_column,
    #height='136px'
)
form_item_layout_tab = Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    justify_content='space-between',
    padding='5px'
)

box_layout_tabs=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=right_column,
    height='100%'
)

button_hide_layout=Layout(
    visibility='hidden', #visible/hidden
    padding='5px',
    
)

button_show_layout=Layout(
    visibility='visible', #visible/hidden
    padding='5px',
    
)

<IPython.core.display.Javascript object>

In [2]:
'''
CSS = """
.output {
    align-items: center;
}
div.output_area {
    width: 100%;
}
div.output_area {
    left-padding: 60px;
}
"""
display(HTML('<style>{}</style>'.format(CSS)))

'''

js_custom="""<script>
$('#appmode-leave').hide();                          // Hides the edit app button.
$('#appmode-busy').hide();                           // Hides the kernel busy indicator.
</script>
"""

display(HTML(js_custom))

In [3]:
%%capture
# This hides any output from this cell
%set_env OMP_NUM_THREADS=4

In [4]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 175px
}
</style>

In [5]:
temperature = widgets.BoundedIntText(
    value=298,
    min=288,
    max=323,
    step=1,
    description='Temperature (K)',
    style=style,
)


In [6]:
bendingModulus = widgets.BoundedIntText(
    value=500,
    min=100,
    max=5000,
    step=1,
    description="Bending Modulus (kB T)",
    style=style
)

In [7]:
stretchingModulus = widgets.BoundedIntText(
    value=500,
    min=100,
    max=5000,
    step=1,
    description="Stretching Modulus (kB T)",
    style=style
)


In [8]:
protein = widgets.BoundedFloatText(
    value=800,
    min=100,
    max=1000,
    step=1,
    description='Protein Concentration (micromolar):',
    style=style
)

In [9]:
LJ = widgets.BoundedFloatText(
    value=1.90,
    min=1,
    max=3,
    step=0.01,
    description='LJ-attraction (kB T):',
    style=style
)

In [10]:
steps = widgets.BoundedIntText(
    value=10000,
    min=10000,
    max=10000000,
    step=10000,
    description='Simulation steps:',
    style=style
)

In [11]:
imageSlider = widgets.IntSlider(
    value=0,
    min=0,
    max=0,
    step=1000,
    description='Images',
    style=style
)        

def onSliderChange(b):
    try:
        index_str = str(imageSlider.value)
        combined_file='snapshot_png/shape_'+index_str+'.png'
                
        #end = time.time()
        #time_taken = end - start
        #localOutput.value = localOutput.value +"loop pre: " + str(time_taken)+"\n"
                
        if os.path.isfile(combined_file):
            file = open(combined_file, "rb")
            image_png = file.read()
            output_image.value= image_png
        else:
            imageSlider.value=imageSlider.value-imageSlider.step
            
    except:
        sys.stderr.write('Image file is not there yet.')
    
imageSlider.observe(onSliderChange, names='value')


In [12]:
submit_str = ''
def onToggleBtnChange(b):
    global submit_str
    if clustorMode.value:
        clustorMode.icon = 'check' 
        clustorMode.button_style='success'
        if not submit_str:
            output_image_warning_text.layout.visibility = 'visible'
    else:
        clustorMode.icon = '' 
        clustorMode.button_style=''
        if not submit_str:
            output_image_warning_text.layout.visibility = 'hidden'

'''
clustorMode =widgets.ToggleButton(
    value=False,
    description='Cluster mode',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulation will run on a computing cluster when this button is turned on but you will loose the access to dynamic simulation snapshot slider.',
    icon='',
    style=style
)
'''

clustorMode =widgets.Checkbox(
    value=False,
    description='Cluster mode',
    disabled=False,
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulation will run on a computing cluster when this button is turned on but you will loose the access to dynamic simulation snapshot slider.',
    icon='',
    style=style
)


clustorMode.observe(onToggleBtnChange,'value')

In [13]:
play_trigger = False

def play_btn_thread():
    global play_trigger
    imageSlider.value = 0
    while play_trigger:
            imageSlider.value = (imageSlider.value + imageSlider.step) % (imageSlider.max + imageSlider.step)
            time.sleep(1)
    
def onPlayBtnChange(b):
    global play_trigger
    global submit_str
    if playMode.value:
        playMode.button_style = 'success'
        play_trigger = True
        t = Thread(target=play_btn_thread)
        t.start()
        
    else:
        imageSlider.value=0
        playMode.button_style = ''
        play_trigger = False
            

playMode =widgets.ToggleButton(
    value=False,
    description='Play snapshots',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulation snapshots will be played as a gif when this button is turned on',
    icon='',
    style=style
)

playMode.observe(onPlayBtnChange,'value')

In [14]:
label_style = "style='background-color: #9EE9E0; font-size:500; padding: 2px'"
label_style2 = "style='font-size:350; padding: 2px; font-weight: bold; align: center'"
def make_label(name, label_style, desc="",):
    lval = '<p %s %s>%s</p>' % (desc, label_style, name)
    return widgets.HTML(lval)

plabel = make_label('Input Parameters', label_style, '')
clabel = make_label('Computing Parameters',label_style, '')
olabel = make_label('Output Controllers', label_style, '')
imageSliderabout = make_label('Slide to navigate the simulation snapshots', label_style2, '')

In [15]:
'''
image_html = widgets.HTML(
    value='<img src="../images/overview.jpg" style="width: 500px"/>',
)
'''
file = open("../images/overview.png", "rb")
image = file.read()
image=widgets.Image(
    value=image,
    format='jpg',
    width=297,
    height=297,
)

form_items = [
    Box([image], layout=form_item_layout)
]

mainImage = Box(form_items, layout=box_layout_image)

In [16]:
form_items = [
    Box([plabel], layout=form_item_layout),
    Box([temperature], layout=form_item_layout),
    Box([bendingModulus], layout=form_item_layout),
    Box([stretchingModulus], layout=form_item_layout),
    Box([protein], layout=form_item_layout),
    Box([LJ], layout=form_item_layout)
]

form_items_computing = [
    Box([clabel], layout=form_item_layout),
    Box([clustorMode], layout=form_item_layout),
    Box([steps], layout=form_item_layout)
]

form_items_output = [
    Box([olabel], layout=form_item_layout),
    Box([imageSlider], layout=form_item_layout),
    Box([imageSliderabout], layout=form_item_layout)
]

physical_para = Box(form_items, layout=box_layout_phisical)
computing_para = Box(form_items_computing, layout=box_layout_computing)
output_para = Box(form_items_output, layout=box_layout_output)

In [17]:
about=widgets.Textarea(
    value='Virus capsids self-assembly spontaneously, often from a single subunit. This app simulates the assembly of Minute Virus of Mice (MVM) proteins into a T=1 capsid. Subunits are coarse-grained trimers based on atomic-resolution data. Temperature, elasticity, protein concentration and LJ-attraction (akin to protein hydrophobicity) are adjustable parameters. You can look at our documtation page for more information about the phase-mapping of these inputs and how they can be adjusted to promote assembly vs. kinetic-trapping behavior. An equilibrium mass spectrum, system energies, and simulation snap shots are produced as outputs. Additional files are provided on the Downloads tab for the raw data of these outputs as well as system parameters.',
    placeholder='',
    layout={'height': '192px','width': '100%'},
    disabled=True,
    style = {'font_weight': '150px'}
)

form_about = [
    Box([about], layout=form_item_layout)
]

about_text = Box(form_about, layout=box_layout_about)


In [18]:
def clear_All():
    global play_trigger
    global how_many_snapshots, previous_steps
    localOutput.value = ''
    submit_str= ''
    progressBar.value=0
    imageSlider.max=0
    imageSlider.value=0 
    #output_image.value=imageEmptyBox
    #output_image.clear_output()
    model_params.layout = movie_download.layout = ms_download.layout = energy_file_1.layout = final_shape_download.layout = button_hide_layout
    clustorMode.disabled = False
    
    playMode.value = False
    how_many_snapshots=0
    play_trigger = False
    previous_steps = 0
    playMode.layout.visibility = 'hidden'
    
    if clustorMode.value:
        output_image_warning_text.layout.visibility = 'visible'
    else:
        output_image_warning_text.layout.visibility = 'hidden'
        
    plt.figure('Mass Spectrum')
    plt.clf()  # clear old plot
    plt.figure('Energy')
    plt.clf()  # clear old plot

In [19]:
def runPreprocessor(npCharge_value, nLigand_value, salt_value):
    '''
    preprocessingCMD = "./preprocessor -Q "+str(npCharge_value)+" -n "+str(nLigand_value)+" -c "+str(salt_value) + " > preprocessor.log"
    
    try:
        !$preprocessingCMD
        
    except  Exception as e:
        localOutput.value = localOutput.value + "Error occured during the execution of preprocessing executable\n"
        sys.stderr.write('Error occured during the execution of preprocessing executable')
        sys.stderr.write(str(e))
        sys.exit(1)
    ''' 
    pass


In [20]:
runFlag=False
working_dir=""
parameter_append=""
sim_bigin=False
build_Plots=True
simulation_params=""

def callExe(s):
    global runFlag, working_dir, parameter_append, sim_bigin, ionsplot, build_Plots, simulation_params, nLigand_value, runName, simulationStepsToatal
    global submit_str
    if build_Plots:
        showPlotUI()
        build_Plots = False
    clear_All()
    
    #clustorMode.disabled = True
    
    localOutput.value = localOutput.value + "Simulation begins...\n"
     
    temperature_value = temperature.value
    bendingModulus_value = bendingModulus.value
    stretchingModulus_value = stretchingModulus.value
    protein_value = protein.value
    LJ_value = LJ.value
    simulationStepsToatal = steps.value
    aneal_freq = int(simulationStepsToatal*0.1)

    total_processors = 96
    omp_size = 24
    
    '''
    following are for 96 processors (4*24) in brown cluster
    20min: 67,500 steps
    for 150,000 steps := 45 mins
    10,000 := 3mins
    '''
    walltime = 5 + int(simulationStepsToatal/10000*3)
    
    runName='souffle'
    
    #Adding standard outputs to string to be sent to output widget callback function
    parameter_append= "Temperature is "+ str(temperature_value)+"\n"
    parameter_append= parameter_append + "Bending Modulus is "+ str(bendingModulus_value)+"\n"
    parameter_append= parameter_append + "Stretching Modulus is "+ str(stretchingModulus_value)+"\n"
    parameter_append= parameter_append + "Protein Concentration is "+ str(protein_value)+"\n"
    parameter_append= parameter_append + "LJ-attraction is "+ str(LJ_value)+"\n"
    
    if clustorMode.value:
        parameter_append= parameter_append + "requested walltime is "+ str(walltime)+" (mins)\n"
        parameter_append= parameter_append + "requested total processors are "+ str(total_processors)+"\n"
    
    simulation_params="_%d" % int(temperature_value)+"_%d" % int(bendingModulus_value)+"_%d" % int(stretchingModulus_value)+\
    "_%.2f" % float(protein_value)+"_%d" % int(LJ_value)

    shutil.rmtree('outfiles',True)
    if not os.path.exists('outfiles'):
        os.makedirs('outfiles')
        
    shutil.rmtree('simulation_snapshots',True)
    if not os.path.exists('simulation_snapshots'):
        os.makedirs('simulation_snapshots')
        
                
    shutil.rmtree('snapshot_png',True)
    if not os.path.exists('snapshot_png'):
        os.makedirs('snapshot_png')    
     
    #ocalOutput.value = localOutput.value + "Preprocessing is running...\n"
    
    #preprocessing stage
    #unPreprocessor(npCharge_value, nLigand_value, salt_value)
    
    #ocalOutput.value = localOutput.value + "Preprocessing completed\n"
        
    localOutput.value = localOutput.value +  parameter_append;
    
    #submit_para_list = '--venue ' + str(cluster_name) + ' -w '+ str(walltime)+' -n '+str(total_processors) \
    # ' -N ' + '16 ' + ' --tailStdout 1000 ' + '--inputfile ' + 'data ' + 'nanoconfinement-r32 '
     
    prog_paraList = ' -f trimer_MVM -S 20 -c 1000 -t 0.001 -K '+str(temperature_value)+' -E '+ str(LJ_value) \
    +' -b '+str(bendingModulus_value)+' -s '+str(stretchingModulus_value)+' -C '+ str(protein_value) \
    + ' -T ' +str(simulationStepsToatal)

    runFlag=True
    sim_bigin=True
    
    computing_clusters = ['standby@brown','standby@rice']
    
    cluster_name = computing_clusters[random.randint(0,1)]
    
    #starting a thread for on the fly density plot
    #t2 = Thread(target=iterative_density_plot)
    #t2.start()
    
    try:
        #rname = s.make_rname(e_np_value, e_out_value, pion_value, c_out_value, np_radius_value, np_charge_value, sim_steps_value)
        submit_str = ''
        if clustorMode.value:
            submit_str = '--venue standby@brown -n '+str(total_processors)+' -N ' +str(omp_size)+' -w '+str(walltime)+' -e OMP_NUM_THREADS=' + str(omp_size) \
            +' --tailStdout 1000 ' +'--inputfile '+'infiles '+'--inputfile '+'outfiles '+'npshapelab-r126'
            
            #submit_str = '-n '+str(total_processors)+' -w '+str(walltime)+' -N 20'+' -e OMP_NUM_THREADS=1'+' --tailStdout 1000 ' +'--inputfile '+'outfiles '+'lammps-31Mar17-parallel -in in.lammps' 
            #submit_str = '--venue standby@rice ' + '-n 32 -w 60 -N 16'+' --tailStdout 1000 ' +'--inputfile '+'outfiles '+'lammps-31Mar17-parallel -in in.lammps'
            localOutput.value = localOutput.value + "Simulation is submitted to the cluster, waiting for response...\n"
        else:
            submit_str = '--local stdbuf -o0 -e0 ./capsid-souffle' + prog_paraList
            
            localOutput.value = localOutput.value + "Simulation is running...\n"

        s.run(runName, submit_str)
        
        time.sleep(1)
        
    except  Exception as e:
        sys.stderr.write('Error occured during the execution of souffle program')
        sys.stderr.write(str(e))
        sys.exit(1)
    
    runFlag = False
    #t2.join()

In [21]:
def convert_SVG_PNG(svg_file_path, png_file_path):
    if os.path.isfile(svg_file_path):
        image_size = image_width//2

        command_ = "inkscape -z -e "+str(png_file_path)+" -w "+str(image_size)+" -h "+str(image_size)+ " "+str(svg_file_path) + " > snapshot_png/svg_png.log"

        try:
            !$command_

        except  Exception as e:
            localOutput.value = localOutput.value + "Error occured while converting SVG to PNG\n"
            sys.stderr.write('Error occured while converting SVG to PNG')
            sys.stderr.write(str(e))
            #sys.exit(1)

In [22]:
def combine_4_PNGs(png1, png2, png3, png4, combined_file):
    '''
    png1='snapshot_png/shape_top_0.png'
    png2='snapshot_png/shape_front_0.png'
    png3='snapshot_png/shape_left_0.png'
    png4='snapshot_png/shape_perspective_0.png'
    combined_file='snapshot_png/shape_0.png'
    combine_4_PNGs(png1, png2, png3, png4, combined_file)
    '''
    if os.path.isfile(png1) and os.path.isfile(png2) and os.path.isfile(png3) and os.path.isfile(png4):
        image_size = image_width

        command_ = "montage "\
        +" -label TOP "+png1 \
        +" -label FRONT "+ png2 \
        +" -label LEFT " +png3 \
        +" -label PERSPECTIVE "+ png4 \
        +" -frame 4 -shadow -geometry "+str(image_size)+"x"+str(image_size)+"+2+2 -pointsize 30 " \
        + combined_file+" > snapshot_png/svg_png.log"
        
        # Resize command
        command_resize = "convert "\
        +combined_file + " -resize " +str(image_size)+"x"+str(image_size) \
        +" "+combined_file 

        try:
            !$command_
            !$command_resize

        except  Exception as e:
            localOutput.value = localOutput.value + "Error occured while creating combined PNG\n"
            sys.stderr.write('Error occured while creating combined PNG')
            sys.stderr.write(str(e))
            #sys.exit(1)

In [23]:
def doneFunc(s, rdir):
    global submit_str
    global how_many_snapshots
    #this is a condtional variable for few things
    submit_str = ''
    
    progressBar.value=int(100)
    
    localOutput.value = localOutput.value + "Simulation execution finished\n" 
    
    submit_btn.but.disabled = True
    clustorMode.disabled = False
    
    localOutput.value = localOutput.value + "Postprocessing is running...\n" 
    
    
    '''
    localOutput.value = localOutput.value + "Postprocessing is running...\n"
    
    #postprocessing stage
    
    postprocessingCMD = "./postprocessor -n "+str(nLigand_value)+" -N 150 > postprocessor.log"
    try:
        !$postprocessingCMD
        
    except  Exception as e:
        localOutput.value = localOutput.value + "Error occured during the execution of postprocessing executable\n"
        sys.stderr.write('Error occured during the execution of postprocessing executable')
        sys.stderr.write(str(e))
        sys.exit(1)
        
    localOutput.value = localOutput.value + "Postprocessing is finished\n"
    '''
    
    #  Plotting the result
    plot_data()
    energy_plot()
    how_many_snapshots=dump_svgs(filepath='outfiles/ovito.lammpstrj', skip_factor=1, current_step=simulationStepsToatal)
    localOutput.value = localOutput.value + "Number of simulation snapshots generated="+str(how_many_snapshots)+"\n"
    localOutput.value = localOutput.value + "Postprocessing is finished\n"
    
    submit_btn.but.disabled = False
    playMode.layout.visibility = 'visible'
    
    #image slider fully ready
    imageSlider.max=int(simulationStepsToatal/imageSlider.step)*imageSlider.step
    #imageSlider.max=(how_many_snapshots-1)*imageSlider.step
    imageSlider.value=int(simulationStepsToatal/imageSlider.step)*imageSlider.step
    #imageSlider.value=int(how_many_snapshots)-1
     
    model_params.layout = movie_download.layout = ms_download.layout = energy_file_1.layout = final_shape_download.layout = button_show_layout

In [24]:
# after the run finished, optionally do something
# with the output
#def plot_data(s, rdir):
def plot_data():
    global build_Plots
    if build_Plots:
        showPlotUI()
        build_Plots = False 
    
    #We ignore rdir because submit puts non-parametric runs in the current directory
    # When caching is enabled, they get put in a chached directory and rdit is always used. 
    #wait for 2 seconds before plotting errorBars.
    time.sleep(0.5)  
    plt.figure('Mass Spectrum')
    plt.clf()  # clear old plot
    #plt.title('Area profile of the nanoparticles')
    plt.xlabel('Cluster Size', fontsize=10)
    plt.ylabel('Weighted cluster number', fontsize=10)
    
    # our simulation writes to outfiles -> density_profile.dat
    try:
        msfile = np.genfromtxt(working_dir + 'outfiles/analysis.ms')
    except:
        sys.stderr.write('Can not find the analysis.ms file')
        sys.exit(1)
    labels = np.arange(len(msfile))
    labels = [x+1 for x in labels]
    plt.bar(labels,msfile,color='purple')
    plt.show()
    
    
def energy_plot():
    global build_Plots
    if build_Plots:
        showPlotUI()
        build_Plots = False 
        
    time.sleep(0.5)
    plt.figure('Energy')
    plt.clf()  # clear old plot
    #plt.title("Energy profiles for the simulation")
    plt.xlabel('Simulation Time Steps', fontsize=10)
    plt.ylabel('Energy (kB T)', fontsize=10)
    
    try:
        
        energy_profile = np.genfromtxt(working_dir + 'outfiles/energy.out')

        #removing Nan rows
        #energy_profile[~np.isnan(energy_profile).any(axis=1)]
        #es_energy_profile[~np.isnan(es_energy_profile).any(axis=1)]

        step = energy_profile[:,0]
        stretching_en = energy_profile[:,2]
        bending_en = energy_profile[:,3]
        lj_en = energy_profile[:,4]
        #potential = energy_profile[:,3]
        #kinetic_virtual = energy_profile[:,4]

        #plt.plot(step,total_en, color='red', label='Global net energy')
        plt.plot(step, stretching_en, color='cyan', label='Stretching energy')
        plt.plot(step, bending_en, color='green', label='Bending energy')
        plt.plot(step,lj_en, color='blue', label='Lennard-Jones potential energy')
        #plt.plot(step,potential, color='blue', label='Potential')
        #plt.plot(step,kinetic_virtual, color='yellow', label='Kinetic (virtual)')
        plt.legend()
        plt.show()
        
    except  Exception as e:
        sys.stderr.write('Can not find the energy file')
        sys.stderr.write(str(e))

In [25]:
views = ['Top', 'Front','Left','Perspective']

def render_pngs(number_of_steps=0):
    
    #start = time.time()
    try:
        index = int(number_of_steps/imageSlider.step)
        image_index = str(number_of_steps)
        index_str = str(index) if index>999 else ( '0'+str(index) if index>99 else ('00'+str(index) if index >9 else '000'+str(index)))
        #print(imageSlider.value)
        #localOutput.value = localOutput.value +"Loop index: " + str(image_index)+"\n"
        
        png1='snapshot_png/shape_'+views[0].lower()+'_'+image_index+'.png'
        png2='snapshot_png/shape_'+views[1].lower()+'_'+image_index+'.png'
        png3='snapshot_png/shape_'+views[2].lower()+'_'+image_index+'.png'
        png4='snapshot_png/shape_'+views[3].lower()+'_'+image_index+'.png'
        combined_file='snapshot_png/shape_'+image_index+'.png'
                
        #end = time.time()
        #time_taken = end - start
        #localOutput.value = localOutput.value +"loop pre: " + str(time_taken)+"\n"
                
        if not os.path.isfile(combined_file):
            threads = []
            for view in views:
                    
                svg_file_path = "simulation_snapshots/image_"+view.lower()+'_'
                png_file_path ="snapshot_png/shape_"+view.lower()+'_'
                ## Convert final image to PNG
                t = Thread(target=convert_SVG_PNG, args=(svg_file_path+index_str+".svg",png_file_path+image_index+".png"))
                threads.append(t)
                t.start()
                #convert_SVG_PNG(svg_file_path+index_str+".svg",png_file_path+image_index+".png")
           
    
            # Wait for all of them to finish
            for x in threads:
                x.join()
            
            combine_4_PNGs(png1, png2, png3, png4, combined_file)

            if number_of_steps == simulationStepsToatal:
                combined_file='snapshot_png/shape_final.png'
                combine_4_PNGs(png1, png2, png3, png4, combined_file)
                        
            
    except  Exception as e:
        sys.stderr.write('There was a error in PNG conversion')
        sys.stderr.write(str(e))
        return 0

def dump_svgs(filepath='outfiles/ovito.lammpstrj', skip_factor=1, current_step=0):
    
    try:
        '''
        shutil.rmtree('simulation_snapshots',True)
        if not os.path.exists('simulation_snapshots'):
            os.makedirs('simulation_snapshots')
        
                
        shutil.rmtree('snapshot_png',True)
        if not os.path.exists('snapshot_png'):
            os.makedirs('snapshot_png')
        '''
        
        data = dump(filepath)
        data.map(1,"id")
        data.tselect.skip(skip_factor)
        t = data.time()
        if len(t)>1:
        
            for view in views:

                theta_ =60
                phi_ = -120
                svg_file_path = "simulation_snapshots/image_"+view.lower()+'_'

                if view is 'Top':
                    theta_ =0
                    phi_ = -90
                elif view is 'Front':
                    theta_ =90
                    phi_ = -90
                elif view is 'Left':
                    theta_ =90
                    phi_ = 180 
                elif view is 'Perspective':
                    theta_ =60
                    phi_ = -120

            
                #print(t)
                #if len(t)>500:
                #    skip_factor = 10
                #    data.tselect.skip(skip_factor)
                #t = data.time()

                s = svg(data)
                s.bg("white")               #set background color (def = "black")
                s.size(image_width//2)               #set image size to NxN
                #s.size(N,M)               #set image size to NxM
                #s.rotate(60,135)               #view from z theta and azimuthal phi (def = 60,30)
                ##### Simular to Ovito UI: Top s.rotate(0,-90), Front s.rotate(90,-90), Left s.rotate(90,180), Perspective s.rotate(60, -120) 
                s.rotate(theta_, phi_) 
                s.shift(0, 0)               #translate by x,y pixels in view window (def = 0,0)
                s.zoom(0.8)               #scale image by factor (def = 1)
                s.box(0)               #0/1/2 = none/variable/fixed box
                #s.box(0/1/2,"green")               #set box color
                #s.box(0/1/2,"red",4)               #set box edge thickness
                s.acol(1,"purple")
                s.acol(2,"white")
                s.acol(5,"green")
                s.acol(6,"purple")
                s.arad(5,1.2)
                s.arad(6,1.8)
                s.file = svg_file_path              #file prefix for created images (def = "image")
                s.all()
                #file_content = s.show(time_index)               #show image of snapshot at timestep N
                #return file_content
                #return SVG(file_content)
                #return display(SVG(file_content))
        
            # Saving as PNGS 
            for number_of_steps in range(0,current_step+1,imageSlider.step):
                render_pngs(number_of_steps)    
            
            imageSlider.max=int(current_step)
            imageSlider.value=int(current_step)   
            return len(t)   # how many images are generated
        else:
            pass

    except  Exception as e:
        sys.stderr.write('Can not find the movie file(ovito.lammpstrj)')
        sys.stderr.write(str(e))
        return 0

In [26]:
in_wrapper_progress = False
saved_data = ""
saved_data_enery = ""
saved_data_ions_pos = ""
stepNumber = 0
previous_steps = 0

def stdoutput_handler(buffer):
    global in_wrapper_progress, saved_data, parameter_append, sim_bigin, stepNumber, saved_data_enery, saved_data_ions_pos, simulationStepsToatal
    global previous_steps
    output = []

    #return buffer

    if sim_bigin:
        #output.append(parameter_append)
        sim_bigin = False
   
    data = buffer.split('\n')
    
    last = len(data) - 1
    
    for i, line in enumerate(data):
        
        number = line.split('%')
        if number:
            if number[0].strip().isdigit():
                #print(number[0].strip())
                percent = float(number[0].strip())     #/simulationStepsToatal*100
                progressBar.value=int(percent)
                # This adjust the image slider dynamically
                # dynamic update only happens for local VM run
                if submit_str.startswith('--local '):
                    number_of_steps =int(int(percent)*simulationStepsToatal/100)
                    
                    # This take care of: if we miss exact % ==0 step from std buffer
                    if (previous_steps+imageSlider.step) < number_of_steps:
                        number_of_steps = int(number_of_steps/imageSlider.step)*imageSlider.step
                    
                    if(number_of_steps%imageSlider.step == 0) and number_of_steps!=previous_steps and number_of_steps!=0:
                        previous_steps=number_of_steps
                        #localOutput.value = localOutput.value + str(number_of_steps)+"\n"
                        #print(number_of_steps)
                        Thread(target=dump_svgs, args=('outfiles/ovito.lammpstrj', 1, number_of_steps)).start()
                        # refreash area and energy plots
                        dynamic_refresh()
            else:
                if '%' not in line and '|' not in line:
                    output.append(line)
      
    reduced_buffer = '\n'.join(output)
    
    #appending new line at the end
    if reduced_buffer:
        if reduced_buffer[-1] != '\n':
            reduced_buffer = reduced_buffer + '\n'
    
    return reduced_buffer

In [27]:
#button_style = "style='background-color: #81c4fd; font-size:500; padding: 4px; -webkit-appearance: button; -moz-appearance: button; appearance: button; text-decoration: none; color: initial;'"

def make_downloadBtn(downFilename="", downloadURL="", displayText=""):
    bval = "<a href='%s' target='_blank' download='%s'><button class='p-Widget jupyter-widgets jupyter-button widget-button mod-info'>%s</button></a>" % (downloadURL, downFilename, displayText)
    return widgets.HTML(bval)

def make_downloadBtn_images(downloadURL="", displayText=""):
    bval = "<a href='%s' target='_blank'><button class='p-Widget jupyter-widgets jupyter-button widget-button mod-info'>%s</button></a>" % (downloadURL, displayText)
    return widgets.HTML(bval)

#from IPython.display import FileLink, FileLinks
#FileLinks('outfiles')
#FileLinks('data')
#download='%s'

In [28]:
%%capture

tab_layout=Layout(
    width='100%',
    height='640px',
    justify_content='space-between',
)

# we want the plot to go here, so create an empty one initially
#create tabs
figsize_tup=(8.50, 5.0)
plt.ioff() #LINE ADDED, Interactive plot blur fix
figure_ms = plt.figure(num='Mass Spectrum', figsize=figsize_tup)
out_ms = Output(layout=tab_layout)

figure_energy = plt.figure(num='Energy', figsize=figsize_tup)
out_energy = Output(layout=tab_layout)

plt.ion() #LINE ADDED

'''
#file = open("../images/image.emptybox.jpg", "rb")
file = open("simulation_snapshots/image0000.svg", "rb")
imageEmptyBox = file.read()
output_image=widgets.Image(
    format='svg',
    width=584,
    height=584,
    value=imageEmptyBox.decode('utf-8')
)
#out_output_image = Output(layout=tab_layout)
'''

#output_image = Output(layout=tab_layout)
file = open("../images/shape_0.png", "rb")
first_frame = file.read()

output_image=widgets.Image(
    format='png',
    width=image_width,
    #height=584,
    value=first_frame
)
#out_output_image = Output(layout=tab_layout)

output_image_layout=Layout(
    display='flex',
    flex_flow='row',
    #border='solid 2px',
    align_items='stretch',
    margin = '0px 0px 0px 80px' #top, right, bottom and left
)

box_layout_warning_text=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 1px',
    align_items='stretch',
    padding='5px',
    width='150px',
    visibility = 'hidden',
    height='95%'
)
box_layout_playbtn=Layout(
    display='flex-end',
    flex_flow='row',
    #border='solid 1px',
    align_items='stretch',
    padding='5px',
    width='150px',
    visibility = 'hidden',
    #height='136px'
)

#label_style3 = "style='background-color: #eeeeee; color: #388e3c; font-size:350; padding: 2px; font-weight: bold; align: center'"
label_style3 = "style='background-color: #eeeeee; font-size:350; padding: 2px; font-weight: bold; align: center'"
output_image_warning_text = make_label('Cluster mode is on! You will lose access to the dynamic simulation snapshot data. Snapshot data will be available at the end of simulation.', label_style3, '')

box_layout_for_right = Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    #border='solid',
    height='100%',
    #width='150px',
    #width='50%'
)

form_output_image_sec_right_items = [
    Box([output_image_warning_text], layout=box_layout_warning_text),
    Box([playMode], layout=box_layout_playbtn),

]

form_output_image_sec_right = Box(form_output_image_sec_right_items, layout=box_layout_for_right)

output_image_box =  Box([output_image], layout=output_image_layout)

form_output_image_sec = [
    Box([output_image_box], layout=output_image_layout),
    Box([form_output_image_sec_right], layout=box_layout_for_right),

]

output_image_section = Box(form_output_image_sec, layout=tab_layout)

#data download tab
final_shape_download = make_downloadBtn_images(downloadURL="snapshot_png/shape_final.png", displayText="Final Configuration")
movie_download = make_downloadBtn(downFilename="movie.xyz", downloadURL="outfiles/ovito.lammpstrj", displayText="Movie")
ms_download = make_downloadBtn(downFilename="ms.dat", downloadURL="outfiles/analysis.ms", displayText="Mass Spectrum")
energy_file_1 = make_downloadBtn(downFilename="energy.dat", downloadURL="outfiles/energy.out", displayText="Energy")
model_params = make_downloadBtn(downFilename="simulation_conditions.dat", downloadURL="outfiles/model.parameters.out", displayText="Simulation Conditions")


model_params.layout = movie_download.layout = ms_download.layout = energy_file_1.layout = final_shape_download.layout = button_hide_layout

box_layout_outputs=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    padding='5px'
)
form_output_content = [
    Box([final_shape_download], layout=box_layout_outputs),
    Box([movie_download], layout=box_layout_outputs),
    Box([ms_download], layout=box_layout_outputs),
    Box([energy_file_1], layout=box_layout_outputs),
    Box([model_params], layout=box_layout_outputs)
    
]
downloadMovieSec = Box(form_output_content, layout=tab_layout)




In [29]:
#tabChange detection and plot data incase automatically not drawn
def dynamic_refresh():
    if os.path.isfile('outfiles/analysis.ms'):
        plot_data()
    if os.path.isfile('outfiles/energy.out'):
        energy_plot() 

def onTabChange(b):
    global build_Plots
    if not build_Plots:
        dynamic_refresh()
        

In [30]:
tab_contents = ['Mass Spectrum', 'Elastic & Binding Energy', 'Simulation Snapshot', 'Downloads']
children = [out_ms, out_energy, output_image_section, downloadMovieSec]
tabSpace = widgets.Tab()
tabSpace.children = children

for i in range(len(children)):
    tabSpace.set_title(i, str(tab_contents[i]))
tabSpace.selected_index = 2

tabSpace.observe(onTabChange, names='selected_index')

form_about = [
    Box([tabSpace], layout=form_item_layout_tab)
]

tab = Box(form_about, layout=box_layout_tabs)

In [31]:
def showPlotUI():
    with out_ms:
        display(figure_ms, layout=tab_layout)
    with out_energy:
        display(figure_energy, layout=tab_layout)

In [32]:
form_item_layout_output = Layout(
    display='flex',
    flex_flow='column',
    justify_content='space-between',
    padding='4px'
)

form_item_layout_run_btn = Layout(
    display='flex',
    flex_flow='column',
    justify_content='space-between',
    padding='4px'
)

#Submit(start_func=callExe, done_func=plot_data, cachename='SubmitTest1')
submit_btn=Submit(start_func=callExe, done_func=doneFunc, outcb=stdoutput_handler)

submit_btn.acc.set_title(0, "Executable Output")
submit_btn.w.layout = form_item_layout_run_btn
#submit_btn.acc.layout = form_item_layout_output
localOutput = widgets.Textarea(layout={'width': '100%', 'height': '250px'})
localOutputWidget = widgets.Accordion(children=[localOutput],layout=form_item_layout_output)
localOutputWidget.set_title(0, "Output")
localOutputWidget.selected_index = None
localOutput.value = ""


In [33]:
progressBar=widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Progress:',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
    layout={'width': '100%'},
    style=style
)

form_items_progress = [
    Box([progressBar], layout=form_item_layout)
]

progressBar_Ele = Box(form_items_progress, layout=box_layout_progress)

In [34]:
app_height='1020px'

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    padding='5px'
)

box_layout_column_left=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=left_column,
    height=app_height
    #height='840px'
)
box_layout_column_right=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=right_column,
    height=app_height
    #height='840px'
)

box_layout=Layout(
    display='flex',
    flex_flow='row',
    #border='solid 2px',
    align_items='stretch',
    width=app_width,
    #padding='5px',
    height=app_height
)

box_layout_rows=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=app_width,
    #padding='5px',
    height='2200px',
    #overflow_x='scroll',
    #overflow_y='scroll'
)


gui_left = [
    Box([mainImage], layout=form_item_layout),
    Box([physical_para], layout=form_item_layout),
    Box([computing_para], layout=form_item_layout),
    Box([progressBar_Ele], layout=form_item_layout),
    Box([output_para], layout=form_item_layout)
]

gui_right = [
    Box([about_text], layout=form_item_layout),
    Box([tab], layout=form_item_layout)
]

form_left = Box(gui_left, layout=box_layout_column_left)
form_right = Box(gui_right, layout=box_layout_column_right)

gui_upper = [
    Box([form_left], layout=box_layout_column_left),
    Box([form_right], layout=box_layout_column_right)
]

form = Box(gui_upper, layout=box_layout)

gui = widgets.VBox([form, localOutputWidget, submit_btn.w],layout=box_layout_rows)


In [35]:
gui

VBox(children=(Box(children=(Box(children=(Box(children=(Box(children=(Box(children=(Box(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x018\x00\x00\x018\x08\x06\x00\x00\x00\xcf\x7f\xefj\x00\x00\x00\x87zTXtRaw profile type exif\x00\x00x\xdau\x8e\xc1\r\xc40\x08\x04\xffT\x91\x120\xe0\x05\xca9E\x89\x94\x0e\xae\xfc\xc3\xb2\xa3\xe4s\xf3\x80\xd5\n\x16\xe8\xf8^\'m\x83\xc6B\xd6=\x90\x00\x17\x96\x96\xf2)\x11<Q\xe6&\xdcF\xaf:Y][)ylR\x99\x02\x19\xce\xf6\x0c\xda\xf2o\xba"p\xba\xb9\xa3c\xc7.\x95.\x87\x8aF\xd5\xda\xa3\x91\xca\xe3\x8d|]\x8b\xa5\xf4\x8f\x7f\x7f\xf1\x03D\n,-1j<[\x00\x00\n\x06iTXtXML:com.adobe.xmp\x00\x00\x00\x00\x00<?xpacket begin="\xef\xbb\xbf" id="W5M0MpCehiHzreSzNTczkc9d"?>\n<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="XMP Core 4.4.0-Exiv2">\n <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\n  <rdf:Description rdf:about=""\n    xmlns:exif="http://ns.adobe.com/exif/1.0/"\n    xmlns:tiff="http://ns.adobe.com/tiff/1.0/"\n   exif:PixelXDimension="312"\n   exif:PixelYDimension="312"\n   tiff:ImageWidth="312"\n   tiff:ImageHeight="312"\n   tiff:Orientation="1"/>\n </rdf:RDF>\n</x:xmpmeta>\n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                                                                                                    \n                           \n<?xpacket end="w"?>\xb5\xab\xe3\x9a\x00\x00\x00\x04sBIT\x08\x08\x08\x08|\x08d\x88\x00\x00 \x00IDATx\xda\xec\xbdy\x98\\wy\xe7\xfb9\xb5\xefU\xbd\xef\xab\xf6\xc5\x92-\xc9\x96%[\xd8\x0e\xe0\x1d\x9c\x18;pI.\x01\x9c\x99;I\x08\x0c$\x99\x0c\xb93s\x99\x87L\x92\xb9\xe1\xe2\xd8\xd9&7!\x01B\x1c\x1b\x19\x83Yl\xbc\xc8\xbb\xb5Y\xb2\xd4\x92,Yj-\xad\xeeV\xb7\xd4K\xad\xa7\xb6\xb3\xce\x1fU\xe7\xe8\xd4\xe9\xaa\x96\x0c\xc6\x98\xf8\xbc\xcfs\x9e\xaa\xee\xae\xee\xae\xfa\x9d\xdf\xfb\xfd\xbd\xcb\xf7}_A\xd7u\x1dG\x1cq\xc4\x91\x7f\x83\xe2r\x96\xc0\x11G\x1cq\x00\xce\x11G\x1cq\xc4\x018G\x1cq\xc4\x11\x07\xe0\x1cq\xc4\x11G\x1c\x80s\xc4\x11G\x1c\xf9I\xc4\xe3,\x81#?k\x19\x1d\x1d\xc5\xe5r!\x08\x02\x00\x9a\xa6\x11\n\x85\xe8\xee\xeev\x16\xc7\x11\x07\xe0\x1c\xf9\xc5\x92\xf3\xe7\xcf3>>\xce\xf8\xf88\xaa\xaa\xe2\xf1xp

Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #5 (got 11 columns instead of 9)Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #5 (got 11 columns instead of 9)
    Line #6 (got 11 columns instead of 9)Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #

simulation_snapshots/image_perspective_0002.svg:1: parser error : Document is empty

^
simulation_snapshots/image_perspective_0002.svg:1: parser error : Document is empty

^

** (inkscape:37374): WARNING **: Specified document simulation_snapshots/image_perspective_0002.svg cannot be opened (does not exist or not a valid SVG file)


Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #5 (got 11 columns instead of 9)
    Line #6 (got 11 columns instead of 9)
    Line #7 (got 11 columns instead of 9)
    Line #8 (got 11 columns instead of 9)

montage.im6: improper image header `snapshot_png/shape_left_2000.png' @ error/png.c/ReadPNGImage/3694.
montage.im6: improper image header `snapshot_png/shape_perspective_2000.png' @ error/png.c/ReadPNGImage/3694.


Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #5 (got 11 columns instead of 9)
    Line #6 (got 11 columns instead of 9)
    Line #7 (got 11 columns instead of 9)
    Line #8 (got 11 columns instead of 9)
    Line #9 (got 11 columns instead of 9)

montage.im6: improper image header `snapshot_png/shape_top_4000.png' @ error/png.c/ReadPNGImage/3694.
montage.im6: improper image header `snapshot_png/shape_left_4000.png' @ error/png.c/ReadPNGImage/3694.
montage.im6: improper image header `snapshot_png/shape_perspective_4000.png' @ error/png.c/ReadPNGImage/3694.


Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #5 (got 11 columns instead of 9)
    Line #6 (got 11 columns instead of 9)
    Line #7 (got 11 columns instead of 9)
    Line #8 (got 11 columns instead of 9)
    Line #9 (got 11 columns instead of 9)
    Line #10 (got 11 columns instead of 9)Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #5 (got 11 columns instead of 9)
    Line #6 (got 11 columns instead of 9)
    Line #7 (got 11 columns instead of 9)
    Line #8 (got 11 columns instead of 9)
    Line #9 (got 11 columns instead of 9)
    Line #10 (got 11 columns instead of 9)
    Line #11 (got 11 columns instead of 9)Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns inste

convert.im6: improper image header `snapshot_png/shape_6000.png' @ error/png.c/ReadPNGImage/3694.
convert.im6: no images defined `snapshot_png/shape_6000.png' @ error/convert.c/ConvertImageCommand/3049.
montage.im6: improper image header `snapshot_png/shape_front_7000.png' @ error/png.c/ReadPNGImage/3694.
montage.im6: improper image header `snapshot_png/shape_left_7000.png' @ error/png.c/ReadPNGImage/3694.
montage.im6: improper image header `snapshot_png/shape_perspective_7000.png' @ error/png.c/ReadPNGImage/3694.
montage.im6: improper image header `snapshot_png/shape_top_7000.png' @ error/png.c/ReadPNGImage/3694.
montage.im6: improper image header `snapshot_png/shape_front_7000.png' @ error/png.c/ReadPNGImage/3694.
montage.im6: improper image header `snapshot_png/shape_perspective_7000.png' @ error/png.c/ReadPNGImage/3694.
convert.im6: improper image header `snapshot_png/shape_7000.png' @ error/png.c/ReadPNGImage/3694.
convert.im6: no images defined `snapshot_png/shape_7000.png' @ err

Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #5 (got 11 columns instead of 9)
    Line #6 (got 11 columns instead of 9)
    Line #7 (got 11 columns instead of 9)
    Line #8 (got 11 columns instead of 9)
    Line #9 (got 11 columns instead of 9)
    Line #10 (got 11 columns instead of 9)
    Line #11 (got 11 columns instead of 9)
    Line #12 (got 11 columns instead of 9)Can not find the energy fileSome errors were detected !
    Line #2 (got 11 columns instead of 9)
    Line #3 (got 11 columns instead of 9)
    Line #4 (got 11 columns instead of 9)
    Line #5 (got 11 columns instead of 9)
    Line #6 (got 11 columns instead of 9)
    Line #7 (got 11 columns instead of 9)
    Line #8 (got 11 columns instead of 9)
    Line #9 (got 11 columns instead of 9)
    Line #10 (got 11 columns instead of 9)
    Line #11 (got 11 columns instead of 9)
  